# 海洋鱼类数据集增强

本Notebook用于可视化和执行数据增强策略，提升海洋鱼类识别模型的性能。

## 目标
1. 可视化不同的数据增强效果
2. 生成增强后的数据集
3. 分析增强策略对模型训练的影响

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import torch
import torchvision.transforms as transforms
from torchvision.transforms import functional as F
import glob
import shutil
from pathlib import Path

# 设置随机种子
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

print("✅ 所有库导入成功！")

## 1. 数据集探索

首先分析原始数据集的结构和样本分布。

In [ ]:
# 数据集路径
dataset_path = "../dataset"

# 获取所有鱼类类别
classes = []
class_counts = {}

if os.path.exists(dataset_path):
    for class_name in os.listdir(dataset_path):
        class_dir = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_dir):
            # 统计每个类别的图片数量
            images = glob.glob(os.path.join(class_dir, "*.png")) + \
                    glob.glob(os.path.join(class_dir, "*.jpg")) + \
                    glob.glob(os.path.join(class_dir, "*.jpeg"))
            
            if len(images) > 0:
                classes.append(class_name)
                class_counts[class_name] = len(images)

    print(f"🔍 发现 {len(classes)} 个鱼类类别:")
    for i, (class_name, count) in enumerate(sorted(class_counts.items())):
        print(f"{i+1:2d}. {class_name:25s} - {count:3d} 张图片")
    
    print(f"\n📊 数据集统计:")
    print(f"总类别数: {len(classes)}")
    print(f"总图片数: {sum(class_counts.values())}")
    print(f"平均每类: {sum(class_counts.values()) / len(classes):.1f} 张")
    print(f"最多图片: {max(class_counts.values())} 张")
    print(f"最少图片: {min(class_counts.values())} 张")
else:
    print("❌ 数据集路径不存在，请检查路径设置")

## 2. 数据增强策略定义

定义多种数据增强方法，包括几何变换、颜色变换等。

In [ ]:
def apply_augmentations(image, aug_type="all"):
    """
    对单张图片应用数据增强
    
    Args:
        image: PIL Image对象
        aug_type: 增强类型 ("rotation", "flip", "color", "brightness", "contrast", "all")
    
    Returns:
        augmented_image: 增强后的PIL Image对象
    """
    augmented = image.copy()
    
    if aug_type == "rotation" or aug_type == "all":
        # 随机旋转 (-15 to 15 degrees)
        angle = random.uniform(-15, 15)
        augmented = augmented.rotate(angle, expand=True, fillcolor=(255, 255, 255))
    
    elif aug_type == "flip" or aug_type == "all":
        # 随机水平翻转
        if random.random() > 0.5:
            augmented = augmented.transpose(Image.FLIP_LEFT_RIGHT)
    
    elif aug_type == "brightness" or aug_type == "all":
        # 亮度调整
        enhancer = ImageEnhance.Brightness(augmented)
        factor = random.uniform(0.7, 1.3)
        augmented = enhancer.enhance(factor)
    
    elif aug_type == "contrast" or aug_type == "all":
        # 对比度调整
        enhancer = ImageEnhance.Contrast(augmented)
        factor = random.uniform(0.8, 1.2)
        augmented = enhancer.enhance(factor)
    
    elif aug_type == "color" or aug_type == "all":
        # 色彩饱和度调整
        enhancer = ImageEnhance.Color(augmented)
        factor = random.uniform(0.8, 1.2)
        augmented = enhancer.enhance(factor)
    
    return augmented

# 定义增强策略组合
augmentation_strategies = {
    "original": "原图",
    "rotation": "旋转",
    "flip": "翻转",
    "brightness": "亮度调整", 
    "contrast": "对比度调整",
    "color": "色彩调整"
}

print("✅ 数据增强函数定义完成！")
print("支持的增强策略:")
for key, desc in augmentation_strategies.items():
    print(f"  - {key}: {desc}")

## 3. 增强效果可视化

展示不同增强策略对样本图片的效果。

In [ ]:
# 选择一些样本图片进行可视化
def visualize_augmentations(class_name=None, num_samples=3):
    """
    可视化数据增强效果
    """
    if not classes:
        print("❌ 没有找到可用的类别")
        return
    
    # 如果没有指定类别，随机选择一个
    if class_name is None:
        class_name = random.choice(classes)
    
    if class_name not in classes:
        print(f"❌ 类别 {class_name} 不存在")
        return
    
    # 获取该类别的图片
    class_dir = os.path.join(dataset_path, class_name)
    images = glob.glob(os.path.join(class_dir, "*.png")) + \
             glob.glob(os.path.join(class_dir, "*.jpg")) + \
             glob.glob(os.path.join(class_dir, "*.jpeg"))
    
    if len(images) == 0:
        print(f"❌ 类别 {class_name} 中没有找到图片")
        return
    
    # 随机选择样本图片
    sample_images = random.sample(images, min(num_samples, len(images)))
    
    for img_idx, img_path in enumerate(sample_images):
        print(f"\n🖼️  样本 {img_idx + 1}: {os.path.basename(img_path)}")
        
        # 加载原图
        original_img = Image.open(img_path).convert('RGB')
        
        # 创建子图
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle(f'数据增强效果展示 - {class_name}', fontsize=16)
        
        # 显示原图和各种增强效果
        aug_types = ['original', 'rotation', 'flip', 'brightness', 'contrast', 'color']
        
        for i, aug_type in enumerate(aug_types):
            row = i // 3
            col = i % 3
            
            if aug_type == 'original':
                img_to_show = original_img
            else:
                img_to_show = apply_augmentations(original_img, aug_type)
            
            axes[row, col].imshow(img_to_show)
            axes[row, col].set_title(f'{augmentation_strategies[aug_type]}', fontsize=12)
            axes[row, col].axis('off')
        
        plt.tight_layout()
        plt.show()

# 执行可视化
if classes:
    # 选择第一个类别进行演示
    demo_class = classes[0]
    print(f"🎯 演示类别: {demo_class}")
    visualize_augmentations(demo_class, num_samples=1)
else:
    print("❌ 无法进行可视化，未找到数据集")

## 4. 生成增强数据集

为每个类别生成增强后的图片，扩充训练数据集。

In [ ]:
def generate_augmented_dataset(
    source_dir="../dataset", 
    output_dir="../augmented_dataset",
    target_samples_per_class=300,
    augmentations_per_image=3
):
    """
    生成增强数据集
    
    Args:
        source_dir: 原始数据集目录
        output_dir: 输出目录
        target_samples_per_class: 每个类别的目标样本数
        augmentations_per_image: 每张原图生成的增强图数量
    """
    
    if not os.path.exists(source_dir):
        print(f"❌ 源目录不存在: {source_dir}")
        return
    
    # 创建输出目录
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    print(f"🚀 开始生成增强数据集...")
    print(f"📁 源目录: {source_dir}")
    print(f"📁 输出目录: {output_dir}")
    print(f"🎯 目标: 每类 {target_samples_per_class} 张图片")
    print(f"🔄 每张原图生成 {augmentations_per_image} 张增强图")
    
    total_generated = 0
    
    for class_name in classes:
        print(f"\n📂 处理类别: {class_name}")
        
        # 创建类别输出目录
        class_output_dir = os.path.join(output_dir, class_name)
        Path(class_output_dir).mkdir(parents=True, exist_ok=True)
        
        # 获取原始图片
        class_input_dir = os.path.join(source_dir, class_name)
        original_images = glob.glob(os.path.join(class_input_dir, "*.png")) + \
                         glob.glob(os.path.join(class_input_dir, "*.jpg")) + \
                         glob.glob(os.path.join(class_input_dir, "*.jpeg"))\n        
        if len(original_images) == 0:
            print(f"  ⚠️ 跳过 {class_name}，没有找到图片")
            continue
        
        print(f"  📊 原始图片: {len(original_images)} 张")
        
        generated_count = 0
        
        # 首先复制原始图片
        for i, img_path in enumerate(original_images):
            if generated_count >= target_samples_per_class:
                break
                
            # 复制原图
            img_name = f"{class_name}_original_{i:04d}.png"
            output_path = os.path.join(class_output_dir, img_name)
            
            original_img = Image.open(img_path).convert('RGB')
            original_img.save(output_path)
            generated_count += 1
            
            # 生成增强图片
            for aug_idx in range(augmentations_per_image):
                if generated_count >= target_samples_per_class:
                    break
                
                # 随机选择增强策略
                aug_strategies = ['rotation', 'flip', 'brightness', 'contrast', 'color']
                selected_aug = random.choice(aug_strategies)
                
                # 应用增强
                augmented_img = apply_augmentations(original_img, selected_aug)
                
                # 保存增强图片
                aug_img_name = f"{class_name}_aug_{selected_aug}_{i:04d}_{aug_idx:02d}.png"
                aug_output_path = os.path.join(class_output_dir, aug_img_name)
                augmented_img.save(aug_output_path)
                generated_count += 1
        
        print(f"  ✅ 生成图片: {generated_count} 张")
        total_generated += generated_count
    
    print(f"\n🎉 数据增强完成!")
    print(f"📊 总计生成: {total_generated} 张图片")
    print(f"📁 保存位置: {output_dir}")
    
    return output_dir

# 配置参数
AUGMENTED_OUTPUT_DIR = "../augmented_dataset"
TARGET_SAMPLES = 200  # 每个类别目标样本数
AUG_PER_IMAGE = 2     # 每张原图生成的增强图数量

print("⚙️  数据增强配置:")
print(f"   输出目录: {AUGMENTED_OUTPUT_DIR}")
print(f"   每类目标样本数: {TARGET_SAMPLES}")
print(f"   每张原图增强数: {AUG_PER_IMAGE}")
print("\n如需开始生成增强数据集，请运行下一个单元格")

In [ ]:
# 执行数据增强（取消注释下面的代码来执行）
# 警告：这将生成大量文件，确保有足够的磁盘空间

# 执行数据增强
if classes and len(classes) > 0:
    print("🚀 开始执行数据增强...")
    
    # 生成增强数据集
    augmented_dir = generate_augmented_dataset(
        source_dir=dataset_path,
        output_dir=AUGMENTED_OUTPUT_DIR,
        target_samples_per_class=TARGET_SAMPLES,
        augmentations_per_image=AUG_PER_IMAGE
    )
    
    print("\n📈 增强数据集统计:")
    if os.path.exists(augmented_dir):
        for class_name in os.listdir(augmented_dir):
            class_dir = os.path.join(augmented_dir, class_name)
            if os.path.isdir(class_dir):
                count = len([f for f in os.listdir(class_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
                print(f"  {class_name}: {count} 张图片")
    
else:
    print("❌ 无法执行数据增强，请检查数据集路径")

## 5. PyTorch数据增强集成

将增强策略集成到PyTorch训练管道中，实现动态数据增强。

In [ ]:
# 定义适用于训练的PyTorch变换
def get_training_transforms():
    """
    获取训练时的数据增强变换
    """
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(
            brightness=0.2,
            contrast=0.2,
            saturation=0.2,
            hue=0.1
        ),
        transforms.RandomAffine(
            degrees=0,
            translate=(0.1, 0.1),
            scale=(0.9, 1.1)
        ),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

def get_validation_transforms():
    """
    获取验证/测试时的变换（不包含随机增强）
    """
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

# 创建变换实例
train_transform = get_training_transforms()
val_transform = get_validation_transforms()

print("✅ PyTorch数据增强变换已定义")
print("\n🔧 训练变换包括:")
print("  - 图像调整大小 (224x224)")
print("  - 随机水平翻转 (50%概率)")
print("  - 随机旋转 (±15度)")
print("  - 颜色抖动 (亮度、对比度、饱和度、色调)")
print("  - 随机仿射变换 (平移、缩放)")
print("  - 标准化 (ImageNet均值和标准差)")

print("\n🔧 验证变换包括:")
print("  - 图像调整大小 (224x224)")  
print("  - 标准化 (ImageNet均值和标准差)")

# 展示变换效果
if classes and len(classes) > 0:
    print(f"\n🎯 演示变换效果 - 使用类别: {classes[0]}")
    
    # 获取一张样本图片
    class_dir = os.path.join(dataset_path, classes[0])
    sample_images = glob.glob(os.path.join(class_dir, "*.png"))[:1]
    
    if sample_images:
        sample_img_path = sample_images[0]
        original_img = Image.open(sample_img_path).convert('RGB')
        
        # 应用变换并可视化
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # 原图
        axes[0].imshow(original_img)
        axes[0].set_title('原图', fontsize=12)
        axes[0].axis('off')
        
        # 训练变换 (需要转换回PIL显示)
        train_tensor = train_transform(original_img)
        # 反标准化用于显示
        inv_normalize = transforms.Normalize(
            mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
            std=[1/0.229, 1/0.224, 1/0.225]
        )
        train_img_display = inv_normalize(train_tensor)
        train_img_display = torch.clamp(train_img_display, 0, 1)
        axes[1].imshow(train_img_display.permute(1, 2, 0))
        axes[1].set_title('训练变换后', fontsize=12)
        axes[1].axis('off')
        
        # 验证变换
        val_tensor = val_transform(original_img)
        val_img_display = inv_normalize(val_tensor)
        val_img_display = torch.clamp(val_img_display, 0, 1)
        axes[2].imshow(val_img_display.permute(1, 2, 0))
        axes[2].set_title('验证变换后', fontsize=12)
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"🔍 变换后张量形状: {train_tensor.shape}")
else:
    print("❌ 无法演示变换效果，未找到数据集")

## 6. 总结与建议

### 📊 数据增强效果分析

通过本Notebook，我们实现了：

1. **离线数据增强**: 预生成增强图片，扩充数据集规模
2. **在线数据增强**: PyTorch训练时动态应用变换
3. **可视化对比**: 直观展示不同增强策略的效果

### 🎯 训练建议

1. **数据增强策略**:
   - 使用适度的旋转和翻转来增加几何变化
   - 颜色抖动帮助模型适应不同光照条件
   - 避免过度增强导致图片失真

2. **训练优化**:
   - 结合预增强数据集和动态增强
   - 在验证集上不使用随机增强
   - 监控过拟合，适当调整增强强度

3. **模型选择**:
   - 建议使用预训练的ResNet50或EfficientNet
   - 利用迁移学习加速训练
   - 根据数据集大小调整学习率

### 🚀 下一步行动

1. 使用增强后的数据集训练新模型
2. 对比增强前后的模型性能
3. 根据实际效果调整增强参数
4. 考虑使用更高级的增强技术（如MixUp、CutMix等）